# week 7

In [1]:
#Q1 

import numpy as np

# random seed for reproducibility
np.random.seed(0)

# generate data from the question
W = np.random.normal(0, 1, 100000)
X = W + np.random.normal(0, 1, 100000)
Z = np.random.normal(0, 1, 100000)
Y = X + Z + W + np.random.normal(0, 1, 100000)

# define the true error term (the part of Y not explained by X)
u = Z + W + (Y - (X + Z + W))  # but simpler: u = Z + W + noise_Y
# or, if we separate explicitly:
epsilon_Y = Y - (X + Z + W)
u = Z + W + epsilon_Y

# compute correlation
corr = np.corrcoef(X, u)[0, 1]
print(f"Correlation between X and the error term u: {corr:.3f}")


Correlation between X and the error term u: 0.407


In [ ]:
# Q2

# random seed
np.random.seed(0)

# generate data
n = 100000
W = np.random.normal(0, 1, n)
X = W + np.random.normal(0, 1, n)
Z = np.random.normal(0, 1, n)
Y = X + Z + W + np.random.normal(0, 1, n)

# error term when regressing Y on X and Z only:
# u = W + epsilon_Y, where epsilon_Y = Y - (X + Z + W)
epsilon_Y = Y - (X + Z + W)
u = W + epsilon_Y  # same as the "unobserved" part if W is omitted

# c
# ompute correlation between X and the regression error
corr = np.corrcoef(X, u)[0, 1]
print(f"Correlation between X and the error term (u): {corr:.3f}")


Correlation between X and the error term (u): 0.499


In [4]:
# Q3

import pandas as pd

# load
df = pd.read_csv("homework_7.1.csv")
df.head()


,Unnamed: 0,X,W,Z,Y
0,0,1.137055,1.221768,0.327829,1.944532
1,1,-0.112905,0.465835,0.599650,0.655514
2,2,2.077755,1.795414,-0.063393,5.934411
3,3,0.456373,-0.512159,1.177413,-0.188064
4,4,-1.012402,0.080002,-0.275697,-0.533775


In [7]:
!pip install statsmodels

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 28.5 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [statsmodels] [statsmodels]

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [9]:
import statsmodels.api as sm
import numpy as np

# drop the index col
df = df.drop(columns=['Unnamed: 0'], errors='ignore')

# helper function to estimate coefficient of X when W ≈ constant
def estimate_coef_at_W(w_value, bandwidth=0.5):
    """Estimate coefficient of X at values of W near w_value."""
    subset = df[(df['W'] > w_value - bandwidth) & (df['W'] < w_value + bandwidth)]
    X_vars = sm.add_constant(subset[['X', 'Z']])
    model = sm.OLS(subset['Y'], X_vars).fit()
    return model.params['X'], len(subset)

# compute coefficient of X at W ≈ -1, 0, 1
results = {w: estimate_coef_at_W(w) for w in [-1, 0, 1]}
results

{-1: (np.float64(0.990090408694134), 2433),
 0: (np.float64(1.4859822514480032), 3821),
 1: (np.float64(1.9936504417092331), 2407)}

In [12]:
# Q4

# error generator with correlation
def make_error(corr_const, num):
    err = []
    prev = np.random.normal(0, 1)
    for n in range(num):
        prev = corr_const * prev + np.random.normal(0, 1)
        err.append(prev)
    return np.array(err)

# simulation settings
num_obs = 200
num_trials = 500
corr_values = [0.2, 0.5, 0.8]

results = {}

for rho in corr_values:
    betas = []
    se_estimates = []
    
    for _ in range(num_trials):
        # autocorrelated errors
        err_X = make_error(rho, num_obs)
        err_Y = make_error(rho, num_obs)
        
        # treatment and outcome (X and Y)
        X = 0.5 * np.random.normal(0, 1, num_obs) + err_X
        Y = 2 + 3 * X + err_Y  # True β_X = 3
        
        # fit regression WITH intercept
        model = sm.OLS(Y, sm.add_constant(X)).fit()
        
        betas.append(model.params[1])
        se_estimates.append(model.bse[1])
    
    results[rho] = {
        "beta_std": np.std(betas),
        "mean_SE_est": np.mean(se_estimates)
    }

# results
for rho, vals in results.items():
    print(f"corr_const = {rho}:  std(β̂_X) = {vals['beta_std']:.4f},  mean(SE) = {vals['mean_SE_est']:.4f}")


corr_const = 0.2:  std(β̂_X) = 0.0628,  mean(SE) = 0.0640
corr_const = 0.5:  std(β̂_X) = 0.0844,  mean(SE) = 0.0653
corr_const = 0.8:  std(β̂_X) = 0.1349,  mean(SE) = 0.0678


##### hw reflection Qs

In [2]:
!pip install statsmodels

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 26.4 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [statsmodels] [statsmodels]

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [4]:
import numpy as np
import statsmodels.api as sm
import pandas as pd

# random seed
np.random.seed(42)

# sample size
n_samples = 1000

# --- 1. DGP ---

# Z is the confounder.
# Z positively affects both X and Y.
Z = np.random.normal(0, 1, n_samples)

# X is affected by Z.
# corr(X, Z) will be positive.
X = 1.5 * Z + np.random.normal(0, 1, n_samples)

# Y is affected by both X and Z.
# the true coefficient for X is 2.
# the coefficient for the confounder Z is 3.
true_beta_X = 2
true_beta_Z = 3
Y = true_beta_X * X + true_beta_Z * Z + np.random.normal(0, 1, n_samples)

# create df
df = pd.DataFrame({'X': X, 'Y': Y, 'Z': Z})

# --- 2. Fit the Models ---

# add a constant to predictors
X_with_const = sm.add_constant(df[['X', 'Z']])
X_naive_with_const = sm.add_constant(df[['X']])

# model 1: true model (includes confounder Z)
model_true = sm.OLS(df['Y'], X_with_const).fit()

# model 2: naive model (omits confounder Z)
model_naive = sm.OLS(df['Y'], X_naive_with_const).fit()

# --- 3. Compare Results ---

print("--- Model 1 (True Model: Y ~ X + Z) ---")
print(model_true.summary().tables[1])
print("\n")

print("--- Model 2 (Naive Model: Y ~ X) ---")
print(model_naive.summary().tables[1])
print("\n")

# coefficients for X from both models
coef_X_true_model = model_true.params['X']
coef_X_naive_model = model_naive.params['X']

print(f"True effect of X on Y:     {true_beta_X}")
print(f"Estimated effect of X in full model:  {coef_X_true_model:.4f}")
print(f"Estimated effect of X in naive model: {coef_X_naive_model:.4f}")

if coef_X_naive_model > true_beta_X:
    print("\nResult: The correlation is OVERESTIMATED.")
elif coef_X_naive_model < true_beta_X:
    print("\nResult: The correlation is UNDERESTIMATED.")
else:
    print("\nResult: The correlation is neither over/underestimated.")

--- Model 1 (True Model: Y ~ X + Z) ---
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0061      0.031      0.197      0.844      -0.055       0.067
X              1.9898      0.031     63.691      0.000       1.929       2.051
Z              3.0371      0.056     54.650      0.000       2.928       3.146


--- Model 2 (Naive Model: Y ~ X) ---
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0749      0.062     -1.203      0.229      -0.197       0.047
X              3.3901      0.036     94.922      0.000       3.320       3.460


True effect of X on Y:     2
Estimated effect of X in full model:  1.9898
Estimated effect of X in naive model: 3.3901

Result: The correlation is OVERESTIMATED.


In [ ]:
def run_simulation(n_simulations=1000, n_samples=100):
   
    # list to store the p-value for the irrelevant variable W from each run
    p_values_W = []

    print(f"Running {n_simulations} simulations with {n_samples} samples each...")

    # --- Run Simulation Loop ---
    for i in range(n_simulations):
        # 1. generate data
        # W is just random noise and has no relationship with Y.
        # its true coefficient is 0.
        W = np.random.normal(0, 1, n_samples)
        
        # X is a predictor that !does! have a true relationship with Y.
        X = np.random.normal(0, 1, n_samples)
        
        # Y depends on X (true coefficient = 2) and some random noise,
        # but it does NOT depend on W.
        true_beta_X = 2
        Y = true_beta_X * X + np.random.normal(0, 1, n_samples)
        
        # df
        df = pd.DataFrame({'X': X, 'Y': Y, 'W': W})
        
        # 2. linear regression analysis
        # model: Y = beta_0 + beta_1*X + beta_2*W
        predictors = sm.add_constant(df[['X', 'W']])
        model = sm.OLS(df['Y'], predictors).fit()
        
        # 3. extract and store p-value for coefficient of W
        p_val_W = model.pvalues['W']
        p_values_W.append(p_val_W)

    # --- Report Results ---

    # find best (smallest) p-value
    min_p_value = min(p_values_W)

    # count false positives
    count_significant = sum(1 for p in p_values_W if p < 0.05)

    print("\n--- Results ---")
    print(f"Best p-value found for W's coefficient: {min_p_value:.6f}")
    print(f"Number of times the p-value for W was < 0.05: {count_significant} out of {n_simulations}")
    
    percentage_false_positives = (count_significant / n_simulations) * 100
    print(f"Percentage of 'false positives' (Type I errors): {percentage_false_positives:.1f}%")


if __name__ == '__main__':
    # random seed for reproducibility
    np.random.seed(42)
    run_simulation(n_simulations=1000, n_samples=100)

Running 1000 simulations with 100 samples each...

--- Results ---
Best p-value found for W's coefficient: 0.000475
Number of times the p-value for W was < 0.05: 51 out of 1000
Percentage of 'false positives' (Type I errors): 5.1%
